In [1]:
import numpy
import codecs
import sys

import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


# Data Preparation

In [3]:
train = pd.read_csv('train_task1_latest.csv', encoding='utf-8')
for i,q in enumerate(train.question.values):
    if i<5:
        print(q)

С какого года Русское Царство перешло на летоисчисление от Рождества Христова и празднование Нового года 1 января, а не 1 сентября?
Кто упомянул о его первых разногласиях со Штейнбергом: Педагог хотел видеть в нём продолжателя русской традиции, а не очередного — вслед за Стравинским и Прокофьевым — её разрушителя, композитора с подозрительными модернистскими наклонностями ?
Как называется имеющая мировое значение эпоха в истории культуры Европы, пришедшая на смену Средним векам и предшествующая Просвещению и Новому времени?
Что усугублялось из-за международного давления на правительство Альенде и ограничения торговли с Чили?
Какой характер носят пророчества Леонардо да Винчи?


In [46]:
file = open( 'train_questions.txt', 'w')

for q in list(train.question.values)[:10]:
    try: 
        file.write(q)
    except UnicodeEncodeError:
        pass

file.close()

In [47]:
list(train.question.values[:10])[0]

'С какого года Русское Царство перешло на летоисчисление от Рождества Христова и празднование Нового года 1 января, а не 1 сентября?'

In [48]:
# load ascii text and covert to lowercase
filename = "train_questions.txt"
file = codecs.open(filename,'r','cp1251')
raw_text = file.read()  
raw_text = raw_text.lower()

In [49]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [50]:
signs = ['\n', '\r', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', ':', ';', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xbb', '\xbf', '\xef']

In [51]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  1178
Total Vocab:  39


In [52]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  1078


In [53]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

# Training

In [54]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [55]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [56]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1078/1078 [==============================] - 12s - loss: 3.5259    
Epoch 2/20
1078/1078 [==============================] - 12s - loss: 3.1925    
Epoch 3/20
1078/1078 [==============================] - 12s - loss: 3.1808    
Epoch 4/20
1078/1078 [==============================] - 12s - loss: 3.1749    
Epoch 5/20
1078/1078 [==============================] - 12s - loss: 3.1619    
Epoch 6/20
1078/1078 [==============================] - 11s - loss: 3.1517    
Epoch 7/20
1078/1078 [==============================] - 12s - loss: 3.1512    
Epoch 8/20
1078/1078 [==============================] - 12s - loss: 3.1466    
Epoch 9/20
1078/1078 [==============================] - 12s - loss: 3.1620    
Epoch 10/20
1078/1078 [==============================] - 13s - loss: 3.1514    
Epoch 11/20
1078/1078 [==============================] - 13s - loss: 3.1524    
Epoch 12/20
1078/1078 [==============================] - 13s - loss: 3.1483    
Epoch 13/20
1078/1078 [==========================

# Predict

In [57]:
# load the network weights
filename = "weights-improvement-19-3.1387.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [58]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [70]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print("-"*50)
print("\nDone.")

Seed:
" я мировое значение эпоха в истории культуры европы, пришедшая на смену средним векам и предшествующа "
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         